In [62]:
import numpy as np
import os
from tqdm import tqdm

In [196]:
dataset = 'hcp'
sulcus = 'S.C.-sylv.'
side = 'L'
crops_dir = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{sulcus}/mask/{side}skeleton_rotated.npy'

In [197]:
crops = np.load(crops_dir)
print(crops.shape)

(1114, 61, 25, 22, 1)


In [219]:
nb_samples = 100
list_directions = []
for skel in crops[np.random.choice(crops.shape[0], nb_samples, replace=False),:,:,:,0]:
    # find the average direction between bottom and top
    coords_bottom = np.stack(np.nonzero(skel==30)).T
    coords_top = np.stack(np.nonzero(skel==35)).T
    list_closest_coords_bottom = []
    for coord_top in coords_top:
        idx_closest_bottom = np.argmin(np.sum(np.square(coords_bottom - coord_top),axis=1))
        coord_bottom = coords_bottom[idx_closest_bottom]
        list_closest_coords_bottom.append(coord_bottom)
    closest_coords_bottom = np.stack(list_closest_coords_bottom)
    direction = np.mean(coords_top - closest_coords_bottom, axis=0)
    list_directions.append(direction)
global_direction = np.mean(np.stack(list_directions), axis=0)

100%|██████████| 100/100 [00:00<00:00, 449.96it/s]


In [220]:
print(f'Average vector between tops and bottoms (vx) : {global_direction}')
depth_axis = np.argmax(np.abs(global_direction))
depth_sign = int(np.sign(global_direction[depth_axis]))
print(f'Depth axis : {depth_axis}, orientation : {depth_sign}')
print(f'Corr strength with selected dimension : {np.abs(global_direction[depth_axis]) / np.linalg.norm(global_direction)}')

Average vector between tops and bottoms (vx) : [-0.13665935  1.71561284  0.17888139]
Depth axis : 1, orientation : 1
Corr strength with selected dimension : 0.9915012434655773


In [221]:
# put the depth on last dim, and put the largest between the other two in first dimension
dims = crops.shape[1:4]
order_axes = [0, 1, 2]
order_axes[2], order_axes[depth_axis] = order_axes[depth_axis], order_axes[2]
if dims[order_axes[1]] > dims[order_axes[0]]:
    order_axes[0], order_axes[1] = order_axes[1], order_axes[0]
# rescale to apply to whole array
order_axes = [ax + 1 for ax in order_axes]
order_axes = [0] + order_axes + [4]
print(order_axes)
crops_ordered_dims = np.transpose(crops, axes=order_axes)
# if orientation is negative, flip on this dimension 
if depth_sign == -1:
    crops_ordered_dims = crops_ordered_dims[:,:,:,::-1,:]

[0, 1, 3, 2, 4]


In [222]:
crops_ordered_dims.shape

(1114, 61, 22, 25, 1)